## Algorithm Code

In [1]:
import numpy as np
import random as rnd

# north-west corner allocation
def TP_NWCA(T):
    T_cost = np.copy(T)
    m = T_cost.shape[0]
    n = T_cost.shape[1]

    total_supply = np.sum(T_cost[:m - 1, -1])
    total_demand = np.sum(T_cost[-1, :n - 1])

    if total_supply == total_demand: # balanced
        pass
    elif total_supply < total_demand: # unbalanced + infeasible
        print("Demand > Supply - Solution not possible")
        return None, None
    else: # unbalanced
        dummy_col = np.zeros([m]) # create dummy demand (all cost = 0)
        dummy_col[-1] = total_supply - total_demand

        T_cost = np.insert(T_cost, n - 1, dummy_col, 1) # inserted into tableau
        n += 1

    T_aug = np.copy(T_cost)
    T_aug = T_aug.astype('float64')
    T_sol = np.zeros([m - 1, n - 1]) # solution matrix

    i, j = 0, 0 # start at top left corner of matrix (north west corner)

    while i < m - 1 and j < n - 1:
        di, dj = 0, 0

        if (T_aug[-1, j] > T_aug[i, -1]): # if demand j > supply i
            amnt = T_aug[i, -1] # use all of supply i to demand j
            di = 1 # move down to next supply
        elif (T_aug[-1, j] < T_aug[i, -1]): # else if, demand j < supply i, satisfy all of demand j
            amnt = T_aug[-1, j]
            dj = 1 # move across to next demand
        else: # else , demand j = supply i, satisfy all of demand j using all of supply i
            amnt = T_aug[-1, j]
            di = 1 # move down to next supply
            dj = 1 # move across to next demand

        T_sol[i, j] = amnt # assign amount to solution table

        T_aug[-1, j] -= amnt
        T_aug[i, -1] -= amnt

        i += di
        j += dj

    t_cost = 0

    for i in range(0, m - 1):
        for j in range(0, n - 1):
            t_cost += T_cost[i, j] * T_sol[i, j] # multiply cell cost by cell amount and add to total

    return T_sol, t_cost


# least-cost method
def TP_LC(T):
    T_cost = np.copy(T)
    m = T_cost.shape[0]
    n = T_cost.shape[1]

    total_supply = np.sum(T_cost[:m - 1, -1])
    total_demand = np.sum(T_cost[-1, :n - 1])

    if total_supply == total_demand: # balanced
        pass
    elif total_supply < total_demand: # unbalanced + infeasible
        print("Demand > Supply - Solution not possible")
        return None, None
    else: # unbalanced
        dummy_col = np.zeros([m]) # create dummy demand (all cost = 0)
        dummy_col[-1] = total_supply - total_demand

        T_cost = np.insert(T_cost, n - 1, dummy_col, 1) # inserted into tableau
        n += 1

    T_aug = np.copy(T_cost)
    T_aug = T_aug.astype('float64')
    T_sol = np.zeros([m - 1, n - 1]) # solution matrix

    cost_table = np.zeros([(m - 1) * (n - 1), 3])

    for i in range(0, m - 1): # scan transportation tableau and build table of cells and costs
        for j in range(0, n - 1):
            indx = (i * (n - 1)) + j
            cost_table[indx, 0] = T_cost[i, j]
            cost_table[indx, 1] = i
            cost_table[indx, 2] = j

    cost_table = cost_table[cost_table[:, 0].argsort()] # sort by cost (lowest to highest)
    p = 0

    while np.sum(T_aug[:m - 1, -1]) > 0: # total supply not used up
        min_i, min_j = int(cost_table[p, 1]), int(cost_table[p, 2]) # select next (p'th) lowest cost  cell

        if (T_aug[-1, min_j] > T_aug[min_i, -1]): # if demand j > supply i, use all of supply i to demand j
            amnt = T_aug[min_i, -1]
        elif (T_aug[-1, min_j] < T_aug[min_i, -1]): # else, if demand j < supply i, satisfy all of demand j
            amnt = T_aug[-1, min_j]
        else: # else, demand j = supply i (eliminate both)
            amnt = T_aug[-1, min_j]

        T_sol[min_i, min_j] = amnt # assign amount to solution table

        T_aug[-1, min_j] -= amnt # decrement demand by amount
        T_aug[min_i, -1] -= amnt # decrement supply by amount

        p += 1

    t_cost = 0 # total cost

    for i in range(0, m - 1):
        for j in range(0, n - 1):
            t_cost += T_cost[i, j] * T_sol[i, j] # multiply cell cost by cell amount and add to total cost

    return T_sol, t_cost

# vogel's approximation method
def TP_VAM(T):
    T_cost = np.copy(T)
    m = T_cost.shape[0]
    n = T_cost.shape[1]

    total_supply = np.sum(T_cost[:m - 1, -1])
    total_demand = np.sum(T_cost[-1, :n - 1])

    if total_supply == total_demand:
        pass
    elif total_supply < total_demand:
        print("Demand > Supply - Solution not possible")
        return None, None
    else:
        dummy_col = np.zeros([m]) # create dummy demand (all cost = 0)
        dummy_col[-1] = total_supply - total_demand

        T_cost = np.insert(T_cost, n - 1, dummy_col, 1) # inserted into tableau
        n += 1

    pen_col = np.zeros([m])
    pen_row = np.zeros([n + 1])

    T_aug = np.copy(T_cost)

    T_aug = np.insert(T_aug, n - 1, pen_col, 1) # insert penalty rows
    T_aug = np.insert(T_aug, m - 1, pen_row, 0)

    T_aug = T_aug.astype('float64')
    T_sol = np.zeros([m - 1, n - 1]) # solution matrix

    while np.sum(T_aug[:m - 1, -1]) > 0: # total supply not used up

        for i in range(0, m - 1): # get all penalties for supplies (rows)
            sorted_vals = np.sort(T_aug[i, :n - 1].flatten())

            if sorted_vals[0] == float('inf') and sorted_vals[1] == float('inf'): # if smallest/2nd smallest values inf (row fully done)
                pen = -1
            elif sorted_vals[1] == float('inf'): # if 2nd smallest value 'inf' - only 1 cell left
                pen = 1
            else: # neither smallest/2nd smallest values inf - calc penalty as normal
                pen = sorted_vals[1] - sorted_vals[0]

            T_aug[i, -2] = pen

        for j in range(0, n - 1): # get all penalties for demands (columns)
            sorted_vals = np.sort(T_aug[:m - 1, j].flatten())

            if sorted_vals[0] == float('inf') and sorted_vals[1] == float('inf'): # if smallest/2nd smallest values inf (column fully done)
                pen = -1
            elif sorted_vals[1] == float('inf'): # if 2nd smallest value 'inf' - only 1 cell left
                pen = 1
            else: # neither smallest/2nd smallest values inf - calc penalty as normal
                pen = sorted_vals[1] - sorted_vals[0]

            T_aug[-2, j] = pen

        max_row_penalty = max(T_aug[:-2, -2]) # max row and column penalties
        max_col_penalty = max(T_aug[-2, :-2])

        if max_row_penalty > max_col_penalty: # if max penalty in one of rows, select cell in supply row with min cost
            max_i_indx = np.where(T_aug[:-2, -2] == max_row_penalty)[0][0]

            s_costs = T_aug[max_i_indx, :n - 1]
            min_cost = min(s_costs)
            max_j_indx = np.where(s_costs == min_cost)[0][0]
        elif max_row_penalty < max_col_penalty: # if max penalty in one of cols, select cell in demand col with min cost
            max_j_indx = np.where(T_aug[-2, :-2] == max_col_penalty)[0][0]

            d_costs = T_aug[:m - 1, max_j_indx]
            min_cost = min(d_costs)
            max_i_indx = np.where(d_costs == min_cost)[0][0]
        else: # if penalties equal, select whichever cell can have more assigned
            max_i_indx = np.where(T_aug[:-2, -2] == max_row_penalty)[0][0]
            max_j_indx = np.where(T_aug[-2, :-2] == max_col_penalty)[0][0]

            s_costs = T_aug[max_i_indx, :n - 1] # row of penalised supply costs
            d_costs = T_aug[:m - 1, max_j_indx] # col of penalised demand costs

            min_cost_s = min(s_costs)
            min_cost_d = min(d_costs)

            min_cost_i_indx = np.where(d_costs == min_cost_d)[0][0] # find indices of lowest costs
            min_cost_j_indx = np.where(s_costs == min_cost_s)[0][0]

            min_cost_i_supply = T_aug[min_cost_i_indx, -1] # check demand vs supply (penalised row)
            min_cost_i_demand = T_aug[-1, max_j_indx]

            min_cost_j_supply = T_aug[max_i_indx, -1] # check demand vs supply (penalised col)
            min_cost_j_demand = T_aug[-1, min_cost_j_indx]

            if min(min_cost_j_supply, min_cost_j_demand) > min(min_cost_i_supply, min_cost_i_demand):
                max_i_indx = max_i_indx
                max_j_indx = min_cost_j_indx
            else:
                max_i_indx = min_cost_i_indx
                max_j_indx = max_j_indx

        if (T_aug[-1, max_j_indx] > T_aug[max_i_indx, -1]): # if demand j > supply i, use entire supply
            amnt = T_aug[max_i_indx, -1]

            for k in range(0, n - 1): # eliminate all costs from supply row from further consideration
                T_aug[max_i_indx, k] = 'inf'
        elif (T_aug[-1, max_j_indx] < T_aug[max_i_indx, -1]): # if demand j < supply i, satisfy entire demand
            amnt = T_aug[-1, max_j_indx]

            for k in range(0, m - 1): # eliminate all costs from demand col from further consideration
                T_aug[k, max_j_indx] = 'inf'
        else:
            amnt = T_aug[-1, max_j_indx]

            for k in range(0, n - 1): # eliminate all costs from supply row from further consideration
                T_aug[max_i_indx, k] = 'inf'

            for k in range(0, m - 1): # eliminate all costs from demand col from further consideration
                T_aug[k, max_j_indx] = 'inf'

        T_sol[max_i_indx, max_j_indx] = amnt # assign amount to solution table

        T_aug[max_i_indx, -1] -= amnt
        T_aug[-1, max_j_indx] -= amnt

    t_cost = 0 # total cost

    for i in range(0, m - 1):
        for j in range(0, n - 1):
            t_cost += T_cost[i, j] * T_sol[i, j] # multiply cell cost by cell amount and add to total cost

    return T_sol, t_cost

## Helper Functions

In [8]:
def TP_PrintSolution(table):
    m = table.shape[0]
    n = table.shape[1]

    header = "\nSol"

    for j in range(0, n):
        header += "\tD{0}".format(j + 1)

    print(header)

    for i in range(0, m):
        row = "S{0}".format(i + 1)

        for j in range(0, n):
            row += "\t{0}".format(int(table[i, j]))

        print(row)


def TP_FillTable(T_test):

    m = T_test.shape[0] # no. of rows
    n = T_test.shape[1] # no. of cols

    sum1 = rnd.randint(1000, 5000) # total supply/demand = random

    a = rnd.sample(range(1, sum1), m - 2) + [0, sum1] # generate random sample of m size
    list.sort(a) # sort
    supply = [a[i + 1] - a[i] for i in range(len(a) - 1)] # take differences to get m supplies that add to sum1 (total)

    a = rnd.sample(range(1, sum1), n - 2) + [0, sum1] # generate random sample of n size
    list.sort(a) # sort
    demand = [a[i + 1] - a[i] for i in range(len(a) - 1)] # take differences to get n demands that add to sum1 (total)

    for i in range(0, m):
        for j in range(0, n):
            c = rnd.randint(1, 100) # random cost for given cell

            if i == m - 1 and j == n - 1:
                T_test[i, j] = 0 # dummy cell
            elif i == m - 1: # last row
                T_test[i, j] = demand[j] # assign demand
            elif j == n - 1: # last col
                T_test[i, j] = supply[i] # assign supply
            else:
                T_test[i, j] = c # assign cell cost

    return T_test

# Modified Distribution method for optimality
def TP_OptimalCheck(T_cost, T_sol, cost):
    m = T_cost.shape[0] - 1
    n = T_cost.shape[1] - 1

    optimality = ""

    count_fill = np.count_nonzero(T_sol) # no. of assigned entries

    if m + n - 1 != count_fill: # if S + D - 1 != no assigned cells, solution is degenerate (cant check optimality)
        optimality = "Degenerate"
        return optimality

    vec_A = np.zeros([count_fill + 1, m + n])
    vec_b = np.zeros([count_fill + 1])

    indx = 1

    vec_A[0, 0] = 1 # set extra condition (u_i = 0)
    vec_b[0] = 0

    for i in range(0, m): # set rest of conditions based on assigned cells
        for j in range(0, n):
            if T_sol[i, j] > 0:
                vec_A[indx, i] = 1 # set coefficient of ui in current row of A = 1
                vec_A[indx, m + j] = 1 # set coefficient of vj in current row of A = 1
                vec_b[indx] = T_cost[i, j] # set value in b vector to cost C_ij

                indx += 1

    vec_uv = np.linalg.solve(vec_A, vec_b) # solve for values of ui & vj
    vec_delta = []

    for i in range(0, m):
        for j in range(0, n):
            if T_sol[i, j] == 0: # unoccupied cell
                vec_delta.append(T_cost[i, j] - (vec_uv[i] + vec_uv[j + m]))

    if min(vec_delta) >= 0: # if minimum is non-negative, then solution is optimal
        optimality = "Optimal"
    else:
        optimality = "Not Optimal"

    return optimality

# run Monte Carlo comparison of methods
def TP_Comparison(iter, min_row, max_row, max_col, fb=True):

    if fb:
        print("n\tSize\t\tNW\t\tLC\t\tVA\tLowest")

    n_nwca = 0 # counts
    n_lc = 0
    n_vam = 0

    for i in range(0, iter): # run n iterations

        # m = 3
        # n = 4

        m = rnd.randint(min_row, max_row)
        n = rnd.randint(m + 1, max_col)

        T_test = np.zeros([m + 1, n + 1]) # create and randomly fill table
        T_test = TP_FillTable(T_test)

        sol1, cost1 = TP_NWCA(T_test) # solve using 3 methods
        sol2, cost2 = TP_LC(T_test)
        sol3, cost3 = TP_VAM(T_test)

        optimal = ""

        min_cost = min([cost1, cost2, cost3]) # figure out which of 3 give best result (can be > 1 method)

        if min_cost == cost1: # NWCA found lowest cost
            if not optimal:
                optimal += "NW"
            else:
                optimal += "/NW"
            n_nwca += 1

        if min_cost == cost2: # LC found lowest cost
            if not optimal:
                optimal += "LC"
            else:
                optimal += "/LC"
            n_lc += 1

        if min_cost == cost3: # VAM found lowest cost
            if not optimal:
                optimal += "VA"
            else:
                optimal += "/VA"
            n_vam += 1

        s_string = "{0} x {1}".format(m, n)

        if fb:
            print("{0}\t{1}\t\t{2}\t\t{3}\t\t{4}\t{5}".format(i + 1, s_string, int(cost1), int(cost2), int(cost3), optimal))

    print("\nResults\t\t\tNW\t\tLC\t\tVA")
    print("{0}\t\t\t{1:.4f}\t\t{2:.4f}\t\t{3:.4f}".format("%", n_nwca / iter, n_lc / iter, n_vam / iter))

## Example 1 - 3 x 4 Problem (Specified)

In [3]:
T1 = np.array([[6, 7, 8, 10, 100], [4, 7, 13, 5, 200], [7, 8, 7, 8, 300], [150, 100, 275, 75, 0]])
sol1a, cost1a = TP_NWCA(T1)
sol1b, cost1b = TP_LC(T1)
sol1c, cost1c = TP_VAM(T1)

check1a = TP_OptimalCheck(T1, sol1a, cost1a)
check1b = TP_OptimalCheck(T1, sol1b, cost1b)
check1c = TP_OptimalCheck(T1, sol1c, cost1c)

TP_PrintSolution(sol1a)
print(cost1a)
print(check1a)

TP_PrintSolution(sol1b)
print(cost1b)
print(check1b)

TP_PrintSolution(sol1c)
print(cost1c)
print(check1c)


Sol	D1	D2	D3	D4
S1	100	0	0	0
S2	50	100	50	0
S3	0	0	225	75
4325.0
Not Optimal

Sol	D1	D2	D3	D4
S1	0	100	0	0
S2	150	0	0	50
S3	0	0	275	25
3675.0
Degenerate

Sol	D1	D2	D3	D4
S1	25	75	0	0
S2	125	0	0	75
S3	0	25	275	0
3675.0
Optimal


## Example 2 - 30 x 40 Problem (Randomly Generated)

In [4]:
T6 = TP_FillTable(np.zeros([30 + 1, 40 + 1]))
sol6a, cost6a = TP_NWCA(T6)
sol6b, cost6b = TP_LC(T6)
sol6c, cost6c = TP_VAM(T6)

check6a = TP_OptimalCheck(T6, sol6a, cost6a)
check6b = TP_OptimalCheck(T6, sol6b, cost6b)
check6c = TP_OptimalCheck(T6, sol6c, cost6c)

print(cost6a)
print(check6a)

print(cost6b)
print(check6b)

print(cost6b)
print(check6c)

229391.0
Not Optimal
59375.0
Not Optimal
59375.0
Not Optimal


## Algorithm Performance Comparison 1 (50 iterations)
Row Height: $5 \le m \le 10$

Row Width: $10 \le n \le 15$

Print each output: True

In [9]:
TP_Comparison(50, 5, 10, 15, True)

n	Size		NW		LC		VA	Lowest
1	9 x 10		321980		209227		209299	LC
2	5 x 13		145444		63201		70287	LC
3	7 x 15		98233		26742		26001	VA
4	8 x 15		99173		42744		37021	VA
5	6 x 13		111769		53742		54995	LC
6	7 x 9		226948		137272		115702	VA
7	8 x 12		34387		28973		23372	VA
8	8 x 15		160442		104553		94042	VA
9	8 x 15		188146		71689		105672	LC
10	5 x 14		122191		77437		71672	VA
11	6 x 7		306256		85985		82133	VA
12	8 x 9		206549		127632		140258	LC
13	5 x 13		98231		55420		55732	LC
14	10 x 12		75864		42757		36838	VA
15	7 x 12		50742		29192		25245	VA
16	8 x 13		82822		55035		45864	VA
17	10 x 14		193095		78955		71258	VA
18	9 x 10		176246		95116		93128	VA
19	5 x 11		111300		61266		56133	VA
20	8 x 11		161682		102008		102306	LC
21	8 x 12		247217		106573		104368	VA
22	6 x 12		157665		85541		81808	VA
23	8 x 12		126616		88964		71351	VA
24	6 x 12		53353		40395		42836	LC
25	7 x 8		174759		111892		109787	VA
26	6 x 9		225534		184508		172530	VA
27	5 x 6		271564		247483		230792	VA
28	9 x 11		206876		93938		84808	

## Algorithm Performance Comparison 2 (5000 iterations)
Row Height: $25 \le m \le 50$

Row Width: $50 \le n \le 75$

Print each output: False

In [6]:
TP_Comparison(5000, 25, 50, 75, False)


Results			NW		LC		VA
%			0.0000		0.1866		0.8134
